In [ ]:
## finish this off on the side!!! 
## next steps build function off the best result

## and test it on the test batch

## give a graph that shows y_actual vs y_predict

## save a model for every parameter you care about --- like flight id or route or 

In [81]:
find_best_value(hi[0])

,mean_r2,mean_MAPE,mean_MSE,std_r2,std_MAPE,std_MSE,score,alpha,intercept,tol,...,combined_score,opti_run,n_estimators,max_depth,min_samples_split,min_samples_leaf,random_state,learning_rate,depth,l2_leaf_reg
6,0.104648,2.868991e+16,68.863592,0.015329,2.888578e+15,12.639494,5,NaN,NaN,NaN,...,61.657179,ran_forest_all_features,119.0,15.0,9.0,9.0,NaN,NaN,NaN,NaN


In [78]:
hi = run_regression_models(X_train_scaled, y_train, featurenames)
    # pull the concat best data
    # pull out the features that were lassoed 
    #OUTPUT ===== return concat_best_data, lassoed_features
    
    ## pull out features that are used.. 

c:\Users\erick\gitrepositories\flight_delays_ML\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.118e+04, tolerance: 4.157e+03
  model = cd_fast.enet_coordinate_descent(
[I 2023-09-05 14:44:02,724] A new study created in memory with name: RidgeRegression Optimization
[I 2023-09-05 14:44:02,848] Trial 0 finished with value: 2.9478064431398707e+18 and parameters: {'alpha': 78.97579899649813, 'fit_intercept': True, 'tol': 0.06823022352232623, 'solver': 'svd'}. Best is trial 0 with value: 2.9478064431398707e+18.
[I 2023-09-05 14:44:02,895] Trial 1 finished with value: 3.014204093877133e+18 and parameters: {'alpha': 17.62876948175698, 'fit_intercept': True, 'tol': 0.04158674419882258, 'solver': 'lsqr'}. Best is trial 0 with value: 2.9478064431398707e+18.
[I 2023-09-05 14:44:02,986] T

0:	learn: 80.7089908	total: 6.01ms	remaining: 895ms
1:	learn: 80.2845386	total: 11.3ms	remaining: 839ms
2:	learn: 79.8980205	total: 16.7ms	remaining: 817ms
3:	learn: 79.6034419	total: 20.9ms	remaining: 761ms
4:	learn: 79.3084061	total: 26.6ms	remaining: 771ms
5:	learn: 79.0476119	total: 31.9ms	remaining: 765ms
6:	learn: 78.7980378	total: 37.6ms	remaining: 767ms
7:	learn: 78.5900652	total: 42.9ms	remaining: 762ms
8:	learn: 78.4449856	total: 48ms	remaining: 751ms
9:	learn: 78.1636145	total: 52.9ms	remaining: 740ms
10:	learn: 77.9472919	total: 56.9ms	remaining: 719ms
11:	learn: 77.8564578	total: 62.8ms	remaining: 723ms
12:	learn: 77.5936126	total: 68.1ms	remaining: 717ms
13:	learn: 77.4768284	total: 72.3ms	remaining: 703ms
14:	learn: 77.2146591	total: 76ms	remaining: 684ms
15:	learn: 77.0620314	total: 81.2ms	remaining: 680ms
16:	learn: 76.9551542	total: 86.5ms	remaining: 677ms
17:	learn: 76.7194724	total: 91.1ms	remaining: 668ms
18:	learn: 76.4893274	total: 96.3ms	remaining: 664ms
19:	lea

[I 2023-09-05 14:46:19,630] Trial 7 finished with value: 81.73680866736335 and parameters: {'learning_rate': 0.09416439608388182, 'depth': 2, 'l2_leaf_reg': 2.529969176499739}. Best is trial 7 with value: 81.73680866736335.


137:	learn: 57.9813208	total: 1.54s	remaining: 134ms
107:	learn: 58.8749839	total: 1.31s	remaining: 509ms
102:	learn: 61.5878048	total: 1.42s	remaining: 646ms
121:	learn: 61.1944009	total: 1.43s	remaining: 328ms
59:	learn: 66.7685123	total: 1.64s	remaining: 2.46s
138:	learn: 57.9483981	total: 1.55s	remaining: 123ms
103:	learn: 58.8127456	total: 1.46s	remaining: 647ms
108:	learn: 58.8493862	total: 1.32s	remaining: 495ms
55:	learn: 70.8059640	total: 1.67s	remaining: 2.8s
122:	learn: 61.1598855	total: 1.44s	remaining: 316ms
103:	learn: 61.5409467	total: 1.43s	remaining: 631ms
104:	learn: 58.7844336	total: 1.47s	remaining: 632ms
109:	learn: 58.8242984	total: 1.33s	remaining: 482ms
123:	learn: 61.1539025	total: 1.45s	remaining: 304ms
104:	learn: 61.4990303	total: 1.44s	remaining: 615ms
0:	learn: 80.7360455	total: 96.2ms	remaining: 14.3s
60:	learn: 66.6881023	total: 1.66s	remaining: 2.42s
105:	learn: 58.7175556	total: 1.48s	remaining: 615ms
0:	learn: 80.6923583	total: 54.5ms	remaining: 8.12s

[I 2023-09-05 14:46:22,539] Trial 0 finished with value: 74.43623143055068 and parameters: {'learning_rate': 0.049427653443201394, 'depth': 4, 'l2_leaf_reg': 2.0458833755773274}. Best is trial 0 with value: 74.43623143055068.


...

[I 2023-09-05 14:46:57,972] Trial 0 finished with value: 71.164949215381 and parameters: {'learning_rate': 0.059447150704605385, 'depth': 8, 'l2_leaf_reg': 1.8647553759443276}. Best is trial 0 with value: 71.164949215381.


48:	learn: 55.7927089	total: 938ms	remaining: 1.93s
50:	learn: 59.5239904	total: 1.69s	remaining: 3.28s
149:	learn: 47.0640197	total: 2.99s	remaining: 0us
66:	learn: 56.2775994	total: 2.17s	remaining: 2.69s
52:	learn: 52.1270218	total: 1.13s	remaining: 2.07s
30:	learn: 57.5222422	total: 660ms	remaining: 2.53s
49:	learn: 55.7036937	total: 962ms	remaining: 1.92s
53:	learn: 51.9673628	total: 1.15s	remaining: 2.04s
68:	learn: 55.9576741	total: 2.23s	remaining: 2.62s
52:	learn: 59.1522954	total: 1.76s	remaining: 3.22s
50:	learn: 55.5405490	total: 993ms	remaining: 1.93s
32:	learn: 57.3371778	total: 701ms	remaining: 2.49s
54:	learn: 51.8650408	total: 1.17s	remaining: 2.02s
55:	learn: 51.8237461	total: 1.18s	remaining: 1.98s
51:	learn: 55.3982949	total: 1.01s	remaining: 1.9s
33:	learn: 57.2437282	total: 721ms	remaining: 2.46s
69:	learn: 55.8577266	total: 2.25s	remaining: 2.58s
56:	learn: 51.7429230	total: 1.2s	remaining: 1.96s
52:	learn: 55.3229214	total: 1.02s	remaining: 1.88s
53:	learn: 58.7

[I 2023-09-05 14:46:59,819] Trial 9 finished with value: 72.98347301775922 and parameters: {'learning_rate': 0.09817495963388571, 'depth': 8, 'l2_leaf_reg': 2.4399024841933756}. Best is trial 0 with value: 71.164949215381.
[I 2023-09-05 14:46:59,894] Trial 1 finished with value: 68.31800183605347 and parameters: {'learning_rate': 0.03885996193025148, 'depth': 8, 'l2_leaf_reg': 1.61083491567621}. Best is trial 1 with value: 68.31800183605347.
[I 2023-09-05 14:47:00,330] Trial 4 finished with value: 67.96811227568551 and parameters: {'learning_rate': 0.03955539161951104, 'depth': 8, 'l2_leaf_reg': 1.7448974132470116}. Best is trial 4 with value: 67.96811227568551.


0:	learn: 63.8687503	total: 13.7ms	remaining: 2.04s
1:	learn: 63.3787216	total: 25.2ms	remaining: 1.87s
2:	learn: 62.9692050	total: 34.4ms	remaining: 1.69s
3:	learn: 62.4952965	total: 48.4ms	remaining: 1.76s
4:	learn: 62.1175382	total: 61.4ms	remaining: 1.78s
5:	learn: 61.7816265	total: 75.7ms	remaining: 1.82s
0:	learn: 63.9334425	total: 15ms	remaining: 2.23s
6:	learn: 61.3928281	total: 90.7ms	remaining: 1.85s
1:	learn: 63.4931924	total: 30.4ms	remaining: 2.25s
7:	learn: 60.8916226	total: 106ms	remaining: 1.88s
2:	learn: 63.1141238	total: 46.4ms	remaining: 2.27s
8:	learn: 60.5552332	total: 122ms	remaining: 1.91s
3:	learn: 62.7378494	total: 63ms	remaining: 2.3s
9:	learn: 60.2388521	total: 137ms	remaining: 1.92s
4:	learn: 62.3909343	total: 77.4ms	remaining: 2.24s
10:	learn: 59.8696008	total: 152ms	remaining: 1.91s
5:	learn: 62.0772896	total: 90.6ms	remaining: 2.17s
11:	learn: 59.4616187	total: 166ms	remaining: 1.91s
6:	learn: 61.7154210	total: 104ms	remaining: 2.12s
12:	learn: 59.2045510

[I 2023-09-05 14:47:02,596] Trial 2 finished with value: 68.23941323581424 and parameters: {'learning_rate': 0.05457964731980136, 'depth': 9, 'l2_leaf_reg': 2.1219031677292852}. Best is trial 4 with value: 67.96811227568551.
[I 2023-09-05 14:47:02,700] Trial 11 finished with value: 67.65247597238786 and parameters: {'learning_rate': 0.046423316359541444, 'depth': 9, 'l2_leaf_reg': 1.7703776354630525}. Best is trial 11 with value: 67.65247597238786.


In [67]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import seaborn as sns
import math
import catboost
from catboost import CatBoostRegressor

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import optuna 

# Feel free to add all the libraries you need
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

from geopy.distance import geodesic

In [68]:
df1 = pd.read_csv('data/ordinalencoded.csv', index_col=0)
df = df1[df1['international_flight']==0]
# define features and target
# if you want to create a polynomial you do so at the start and recreate the dataframe
features = df.columns.tolist()
features.remove('target')
X = df[features]
y = df['target']
featurenames = list(X.columns)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=12, test_size=0.5)
#df1 = pd.read_csv('data/ordinalencoded_expandeddataset.csv', index_col=0)

In [69]:
## scaler is applied outside of function
# standard scaler 
scaler = StandardScaler()
#scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [70]:

## pull the column names immediately they are useful
### new definition
def run_regression_models(x234_array, y234_array, features):
## applies lasso optimization to the data set for feature selection
## you have two trains --- lassoed_train & X_train 
## make sure you make a list of the feature in the lassoed train... 

## how many of each iteration do you want 
    ridge_iter = 30
    ran_forest_iter = 10
    SVR_iter = 1
    dec_tree_iter = 30
    
## apply the other optimizations to both the lassoed X-train adn the x-train
    ## 
    lasso_opti = lasso_optimization(x234_array, y234_array, [0.01, 0.08,0.12, 0.15,1,10])
    # output of lasso optimization is a dataframe of with all the attempts, pull out the best 
    ## min score function???
    lasso_bestvalue = find_best_value(lasso_opti)
    # get the lassoed array and the list of its features
    ## lasso features for later
    [lassoed_features, lassoed_X_Train] = get_lassoed_array(lasso_bestvalue, features)
    
    # Ridge Regression
    ridge_all_features = ridge_optimization(x234_array, y234_array, ridge_iter)
    ridge_lasso_features = ridge_optimization(lassoed_X_Train, y234_array, ridge_iter)
    
    ridge_all_features['opti_run']= 'ridge_all_features'
    ridge_lasso_features['opti_run']= 'ridge_lasso_features'
    
    # RandomForestRegressor_optimization
    ran_forest_all_features = RandomForestRegressor_optimization(x234_array, y234_array, ran_forest_iter)
    ran_forest_lasso_features = RandomForestRegressor_optimization(lassoed_X_Train, y234_array, ran_forest_iter)
    
    ran_forest_all_features['opti_run']= 'ran_forest_all_features'
    ran_forest_lasso_features['opti_run']= 'ran_forest_lasso_features'
    
    # SVRegressor_optimization
    #SVR_all_features = SVRegressor_optimization(x234_array, y234_array, SVR_iter)
    #SVR_lasso_features = SVRegressor_optimization(lassoed_X_Train, y234_array, SVR_iter)
    #
    #SVR_all_features['opti_run']= 'SVR_all_features'
    #SVR_lasso_features['opti_run']= 'SVR_lasso_features'
    
    # DecisionTreeRegressor_optimization
    Dec_Tree_all_features = DecisionTreeRegressor_optimization(x234_array, y234_array, dec_tree_iter)
    Dec_Tree_lasso_features = DecisionTreeRegressor_optimization(lassoed_X_Train, y234_array, dec_tree_iter)
    
    Dec_Tree_all_features['opti_run']= 'Dec_Tree_all_features'
    Dec_Tree_lasso_features['opti_run']= 'Dec_Tree_lasso_features'
    
    # CatBooster_optimization
    catBoost_all_features = catboosterregressor_optimization(x234_array, y234_array, 15, 150)
    catBoost_lasso_features = catboosterregressor_optimization(lassoed_X_Train, y234_array, 15, 150)
    
    catBoost_all_features['opti_run']= 'catBoost_all_features'
    catBoost_lasso_features['opti_run']= 'catBoost_lasso_features'
    
    concat_all_data = pd.concat([
        ridge_all_features, 
        ridge_lasso_features,
        ran_forest_all_features,
        ran_forest_lasso_features,
        #SVR_all_features,
        #SVR_lasso_features,
        Dec_Tree_all_features,
        Dec_Tree_lasso_features,
        catBoost_all_features,
        catBoost_lasso_features
        ])
    
    concat_best_data = pd.concat([
        find_best_value(ridge_all_features), 
        find_best_value(ridge_lasso_features),
        find_best_value(ran_forest_all_features),
        find_best_value(ran_forest_lasso_features),
        #find_best_value(SVR_all_features),
        #find_best_value(SVR_lasso_features),
        find_best_value(Dec_Tree_all_features),
        find_best_value(Dec_Tree_lasso_features),
        find_best_value(catBoost_all_features),
        find_best_value(catBoost_lasso_features)
        ])
    
    
# Elastic net
# XGBoost
# CatBoost
    
    # pull the concat best data
    # pull out the features that were lassoed 
    return concat_best_data, lassoed_features


In [71]:
## USEFUL FUNCTIONS
def find_best_value(output_from_opti):
    bestValue = output_from_opti[output_from_opti['combined_score']==min(output_from_opti['combined_score'])]
    return bestValue

# take list of coefficients - if more than zero 
# keep the name of the poly feature
# keep the non-zero poly features values
def get_lassoed_array(a_lasso_opti, column_names):
    ## pull the list of coefficients -- some funnny stuff needs to be done because of array stuff, basically reset_index
    coefficient_list = list(a_lasso_opti['coefficients'].reset_index()['coefficients'][0]) 

    ### creature a dictionary of features and coefficients and then turn into dataframe
    feature_coefficient_dict = {'feature':[], 'coefficient':[]}
    for (feaI, coeI) in zip(featurenames, coefficient_list): 
        if coeI != 0: 
            feature_coefficient_dict['feature'].append(feaI)
            feature_coefficient_dict['coefficient'].append(coeI)
            
    ## coefficient ----- important for later to maybe pull out 
    coefficient_df = pd.DataFrame(feature_coefficient_dict).sort_values('coefficient', ascending=False)
    feature_selection = feature_coefficient_dict['feature']
    lassoed_dataframe = pd.DataFrame(X_train, columns= column_names)[feature_selection]
    #### with the coefficients create a new np.array to 
    lassoed_x_train = np.array(pd.DataFrame(X_train, columns= column_names)[feature_selection])
    ## output is the coefficient list for the lasso and the new lassoed_x_train
    return [feature_coefficient_dict['feature'], lassoed_x_train]

In [72]:
## feature engineering -- create function to pull out coefficients/features
## add a gradient descent
def lasso_optimization(x123_train, y123_train, alpha_list):
    eval_metrics_dictionary = {'mean_r2':[],'mean_MAPE':[],'mean_MSE':[],'std_r2':[],'std_MAPE':[],'std_MSE':[],'score':[], 'alpha':[], 'coeff_used':[], 'coefficients':[], 'intercept':[]}

    # initialize and train model with (default value) alpha = 0.5
    # for i in [0.1, 1, 10,100,1000, 10000, 33000,66000, 100000, 330000,660000,1000000, 3300000,6600000,10000000,33000000,66000000,100000000]:
    for i in alpha_list:

        model = Lasso(alpha=i, max_iter=int(2000))
        model.fit(x123_train, y123_train)
        
        ## EVALUATION METRICS
        # you should really use cross_val_score rather than testing against the test set
        ## use cross validation for 
        score = cross_validate(model, x123_train, y123_train, scoring=('neg_mean_absolute_percentage_error', 'r2', 'neg_root_mean_squared_error'), cv=3)      
        mean_MAPE = -1*np.mean(score['test_neg_mean_absolute_percentage_error'])
        mean_r2 = np.mean(score['test_r2'])
        mean_MSE = -1*np.mean(score['test_neg_root_mean_squared_error'])
        std_MAPE = np.std(score['test_neg_mean_absolute_percentage_error'])
        std_r2 = np.std(score['test_r2'])
        std_MSE = np.std(score['test_neg_root_mean_squared_error'])
        
        coeff_used = np.sum(model.coef_!=0)
        coefficients = model.coef_
        intercept = model.intercept_
        dicKeys = ['mean_MAPE','mean_r2','mean_MSE','std_MAPE','std_r2','std_MSE','score','alpha','coeff_used', 'coefficients', 'intercept'] 
        dicValues = [mean_MAPE, mean_r2, mean_MSE, std_MAPE, std_r2, std_MSE, len(score), i, coeff_used, coefficients, intercept]
        for (keyi, vali) in zip(dicKeys, dicValues):
            eval_metrics_dictionary[keyi].append(vali)
        #eval_metrics_dictionary['mean_MAPE'].append(mean_MAPE)
        #eval_metrics_dictionary['mean_r2'].append(mean_r2)
        #eval_metrics_dictionary['mean_MSE'].append(mean_MSE)
        #eval_metrics_dictionary['std_MAPE'].append(std_MAPE)
        #eval_metrics_dictionary['std_r2'].append(std_r2)
        #eval_metrics_dictionary['std_MSE'].append(std_MSE)
        #eval_metrics_dictionary['score'].append(len(score))
        #eval_metrics_dictionary['alpha'].append(i)
        #eval_metrics_dictionary['coeff_used'].append(coeff_used)

    df_framework = pd.DataFrame(eval_metrics_dictionary)
    # * by the r2_score that is closest to 1 --- you want to minimise the values of the rest, dont need to include RMSE, you overweight that value
    #df_framework['combined_score'] = abs(abs(1 - df_framework['mean_r2'])*df_framework['mean_MSE']*df_framework['mean_MAPE'])
    df_framework['combined_score'] = abs(1 - df_framework['mean_r2'])*df_framework['mean_MSE']
    return df_framework
#return df_framework.sort_values('combined_score')

In [73]:
def ridge_optimization(x123_train, y123_train, number_of_trials):
    # input to study should be dataset and number of iterations
    eval_metrics_dictionary = {'mean_r2':[],'mean_MAPE':[],'mean_MSE':[],'std_r2':[],'std_MAPE':[],'std_MSE':[],'score':[],'alpha': [], 
                               'intercept' : [], 'tol': [], 
                               'solver':[], 'no_of_coefficients':[]}

    def objective(trial):
        ## HYPER PARAMETER TUNING
        alpha = trial.suggest_float("alpha", 0, 100)
        intercept = trial.suggest_categorical("fit_intercept", [True, False])
        tol = trial.suggest_float("tol", 0.001, 1, log=True)
        solver = trial.suggest_categorical("solver", ["auto", "svd","cholesky", "lsqr", "saga", "sag"])

        ## MODEL SELECTION
        ## Create Model
        model = Ridge(alpha=alpha,fit_intercept=intercept,tol=tol,solver=solver)
        ## Fit Model
        model.fit(x123_train, y123_train)
        
        # Function parameters for cross_val_score; 
        #   - estimator - The model object to use to fit the data
        #   - X - The data to fit the model on
        #   - y - The target of the model
        #   - scoring - The error metric to use
        #   - cv - The number of splits to use
        
        # error score to use; 
        #   - accuracy
        #   - balanced_accuracy
        #   - roc_auc
        #   - f1
        #   - neg_mean_absolute_error
        #   - neg_root_mean_squared_error
        #   - r2
        
        ## EVALUATION METRICS
        # you should really use cross_val_score rather than testing against the test set
        score = cross_validate(model, x123_train, y123_train, scoring=('neg_mean_absolute_percentage_error', 'r2', 'neg_root_mean_squared_error'), cv=3)      
        mean_MAPE = -1*np.mean(score['test_neg_mean_absolute_percentage_error'])
        mean_r2 = np.mean(score['test_r2'])
        mean_MSE = -1*np.mean(score['test_neg_root_mean_squared_error'])
        std_MAPE = np.std(score['test_neg_mean_absolute_percentage_error'])
        std_r2 = np.std(score['test_r2'])
        std_MSE = np.std(score['test_neg_root_mean_squared_error'])
        
        """
        y_pred_train= ridge_regressor.predict(X_test)
            
        Evaluation metrics;
        - R squared
        - MSE
        - RMSE
        - MAPE
        
        infun_MSE = mean_squared_error(y_test,y_pred_train)
        infun_RMSE = math.sqrt(infun_MSE)
        infun_r2 = r2_score(y_test, y_pred_train) 
        infun_MAPE = mean_absolute_percentage_error(y_test, y_pred_train) 
        max_coeff = ridge_regressor.coef_.round(3)
        eval_metrics_dictionary['r2'].append(infun_r2)
        eval_metrics_dictionary['MSE'].append(infun_MSE)
        eval_metrics_dictionary['RMSE'].append(infun_RMSE)
        eval_metrics_dictionary['MAPE'].append(infun_MAPE)
        """
        coeff_used = np.sum(model.coef_!=0)

        eval_metrics_dictionary['mean_MAPE'].append(mean_MAPE)
        eval_metrics_dictionary['mean_r2'].append(mean_r2)
        eval_metrics_dictionary['mean_MSE'].append(mean_MSE)
        eval_metrics_dictionary['std_MAPE'].append(std_MAPE)
        eval_metrics_dictionary['std_r2'].append(std_r2)
        eval_metrics_dictionary['std_MSE'].append(std_MSE)
        eval_metrics_dictionary['score'].append(len(score))
        
        eval_metrics_dictionary['alpha'].append(alpha)
        eval_metrics_dictionary['intercept'].append(intercept)
        eval_metrics_dictionary['tol'].append(tol)
        eval_metrics_dictionary['solver'].append(solver)
        eval_metrics_dictionary['no_of_coefficients'].append(coeff_used)

        return mean_MSE*mean_MAPE*(1-mean_r2)*(1-mean_r2)
        #return mean_MSE

    study2 = optuna.create_study(study_name="RidgeRegression Optimization")
    study2.optimize(objective, n_trials=number_of_trials)
    df_framework = pd.DataFrame(eval_metrics_dictionary)
    df_framework['combined_score'] = abs(1 - df_framework['mean_r2'])*df_framework['mean_MSE']
    # * by the r2_score that is closest to 1 --- you want to minimise the values of the rest, dont need to include RMSE, you overweight that value
    return df_framework

In [74]:
def RandomForestRegressor_optimization(x123_train, y123_train,number_of_trials):
    # input to study should be dataset and number of iterations
    eval_metrics_dictionary = {'mean_r2':[],'mean_MAPE':[],'mean_MSE':[],'std_r2':[],'std_MAPE':[],'std_MSE':[],'score':[],
        'n_estimators': [], 
        'max_depth': [],
        'min_samples_split': [],
        'min_samples_leaf':[]
        }
    #randoforreg_dictionary = {'n_estimators': [], 'max_depth': [],'min_samples_split': [],'min_samples_leaf':[]}

    def objective(trial):
        ## HYPER PARAMETER TUNING
        n_estimators = trial.suggest_int("n_estimators", 10, 200, log=True)
        max_depth = trial.suggest_int("max_depth", 2, 32)
        min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
        min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)

        # Create and fit random forest model
        model = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42,
            )
        model.fit(x123_train, y123_train)
        
        # Function parameters for cross_val_score; 
        #   - estimator - The model object to use to fit the data
        #   - X - The data to fit the model on
        #   - y - The target of the model
        #   - scoring - The error metric to use
        #   - cv - The number of splits to use
        
        # error score to use; 
        #   - accuracy
        #   - balanced_accuracy
        #   - roc_auc
        #   - f1
        #   - neg_mean_absolute_error
        #   - neg_root_mean_squared_error
        #   - r2
        
        ## EVALUATION METRICS
        # you should really use cross_val_score rather than testing against the test set
        score = cross_validate(model, x123_train, y123_train, scoring=('neg_mean_absolute_percentage_error', 'r2', 'neg_root_mean_squared_error'), cv=3)      
        mean_MAPE = -1*np.mean(score['test_neg_mean_absolute_percentage_error'])
        mean_r2 = np.mean(score['test_r2'])
        mean_MSE = -1*np.mean(score['test_neg_root_mean_squared_error'])
        std_MAPE = np.std(score['test_neg_mean_absolute_percentage_error'])
        std_r2 = np.std(score['test_r2'])
        std_MSE = np.std(score['test_neg_root_mean_squared_error'])
        
        """
        y_pred_train= ridge_regressor.predict(X_test)
            
        Evaluation metrics;
        - R squared
        - MSE
        - RMSE
        - MAPE
        
        infun_MSE = mean_squared_error(y_test,y_pred_train)
        infun_RMSE = math.sqrt(infun_MSE)
        infun_r2 = r2_score(y_test, y_pred_train) 
        infun_MAPE = mean_absolute_percentage_error(y_test, y_pred_train) 
        max_coeff = ridge_regressor.coef_.round(3)
        eval_metrics_dictionary['r2'].append(infun_r2)
        eval_metrics_dictionary['MSE'].append(infun_MSE)
        eval_metrics_dictionary['RMSE'].append(infun_RMSE)
        eval_metrics_dictionary['MAPE'].append(infun_MAPE)
        """
        #coeff_used = np.sum(model.coef_!=0)

        eval_metrics_dictionary['mean_MAPE'].append(mean_MAPE)
        eval_metrics_dictionary['mean_r2'].append(mean_r2)
        eval_metrics_dictionary['mean_MSE'].append(mean_MSE)
        eval_metrics_dictionary['std_MAPE'].append(std_MAPE)
        eval_metrics_dictionary['std_r2'].append(std_r2)
        eval_metrics_dictionary['std_MSE'].append(std_MSE)
        eval_metrics_dictionary['score'].append(len(score))
        
        eval_metrics_dictionary['n_estimators'].append(n_estimators)
        eval_metrics_dictionary['max_depth'].append(max_depth)
        eval_metrics_dictionary['min_samples_split'].append(min_samples_split)
        eval_metrics_dictionary['min_samples_leaf'].append(min_samples_leaf)
        #randoforreg_dictionary['no_of_coefficients'].append(coeff_used)

        return mean_MSE*mean_MAPE*(1-mean_r2)*(1-mean_r2)
        #return mean_MSE

    study2 = optuna.create_study(study_name="RandomForestRegression Optimization")
    study2.optimize(objective, n_trials=number_of_trials)
    trial = study2.best_trial
    df_framework = pd.DataFrame(eval_metrics_dictionary)
    # * by the r2_score that is closest to 1 --- you want to minimise the values of the rest, dont need to include RMSE, you overweight that value
    df_framework['combined_score'] = abs(1 - df_framework['mean_r2'])*df_framework['mean_MSE']
    return df_framework


In [75]:

def SVRegressor_optimization(x123_train, y123_train,number_of_trials):
    # input to study should be dataset and number of iterations
    eval_metrics_dictionary = {'mean_r2':[],'mean_MAPE':[],'mean_MSE':[],'std_r2':[],'std_MAPE':[],'std_MSE':[],'score':[],'kernel' :[],'C' :[],'degree' :[],'coef0' :[],'gamma' :[]}
    def objective(trial):
        ## HYPER PARAMETER TUNING
        degree = trial.suggest_int("degree", 3, 8)
        C = trial.suggest_int("C", 1, 10)
        coef0 = trial.suggest_float("coef0", 0.01, 10)
        gamma = trial.suggest_categorical("gamma", ["auto", 'scale'])
        kernel = trial.suggest_categorical("kernel", ['poly'])
        # ['linear', 'poly', 'rbf', 'sigmoid'])


        # Create and fit random forest model
        model = SVR(
            kernel=kernel,
            C=C,
            degree=degree,
            coef0=coef0,
            gamma=gamma,
            )
        model.fit(x123_train, y123_train)
        
        # Function parameters for cross_val_score; 
        #   - estimator - The model object to use to fit the data
        #   - X - The data to fit the model on
        #   - y - The target of the model
        #   - scoring - The error metric to use
        #   - cv - The number of splits to use
        
        # error score to use; 
        #   - accuracy
        #   - balanced_accuracy
        #   - roc_auc
        #   - f1
        #   - neg_mean_absolute_error
        #   - neg_root_mean_squared_error
        #   - r2
        
        ## EVALUATION METRICS
        # you should really use cross_val_score rather than testing against the test set
        score = cross_validate(model, x123_train, y123_train, scoring=('neg_mean_absolute_percentage_error', 'r2', 'neg_root_mean_squared_error'), cv=3)      
        mean_MAPE = -1*np.mean(score['test_neg_mean_absolute_percentage_error'])
        mean_r2 = np.mean(score['test_r2'])
        mean_MSE = -1*np.mean(score['test_neg_root_mean_squared_error'])
        std_MAPE = np.std(score['test_neg_mean_absolute_percentage_error'])
        std_r2 = np.std(score['test_r2'])
        std_MSE = np.std(score['test_neg_root_mean_squared_error'])
        
        """
        y_pred_train= ridge_regressor.predict(X_test)
            
        Evaluation metrics;
        - R squared
        - MSE
        - RMSE
        - MAPE
        
        infun_MSE = mean_squared_error(y_test,y_pred_train)
        infun_RMSE = math.sqrt(infun_MSE)
        infun_r2 = r2_score(y_test, y_pred_train) 
        infun_MAPE = mean_absolute_percentage_error(y_test, y_pred_train) 
        max_coeff = ridge_regressor.coef_.round(3)
        eval_metrics_dictionary['r2'].append(infun_r2)
        eval_metrics_dictionary['MSE'].append(infun_MSE)
        eval_metrics_dictionary['RMSE'].append(infun_RMSE)
        eval_metrics_dictionary['MAPE'].append(infun_MAPE)
        """
        #coeff_used = np.sum(model.coef_!=0)

        eval_metrics_dictionary['mean_MAPE'].append(mean_MAPE)
        eval_metrics_dictionary['mean_r2'].append(mean_r2)
        eval_metrics_dictionary['mean_MSE'].append(mean_MSE)
        eval_metrics_dictionary['std_MAPE'].append(std_MAPE)
        eval_metrics_dictionary['std_r2'].append(std_r2)
        eval_metrics_dictionary['std_MSE'].append(std_MSE)
        eval_metrics_dictionary['score'].append(len(score))
        
        eval_metrics_dictionary['degree'].append(degree)
        eval_metrics_dictionary['C'].append(C)
        eval_metrics_dictionary['coef0'].append(coef0)
        eval_metrics_dictionary['gamma'].append(gamma)
        eval_metrics_dictionary['kernel'].append(kernel)
        #randoforreg_dictionary['no_of_coefficients'].append(coeff_used)
        #return mean_MSE*mean_MAPE*(1-mean_r2)*(1-mean_r2)
        return mean_MSE

    study2 = optuna.create_study(study_name="SVRRegression Optimization")
    study2.optimize(objective, n_trials=number_of_trials)
    df_framework = pd.DataFrame(eval_metrics_dictionary)
    # * by the r2_score that is closest to 1 --- you want to minimise the values of the rest, dont need to include RMSE, you overweight that value
    df_framework['combined_score'] = abs(1 - df_framework['mean_r2'])*df_framework['mean_MSE']
    return df_framework

In [76]:
def DecisionTreeRegressor_optimization(x123_train, y123_train,number_of_trials):
    # input to study should be dataset and number of iterations
    eval_metrics_dictionary = {'mean_r2':[],'mean_MAPE':[],'mean_MSE':[],'std_r2':[],'std_MAPE':[],'std_MSE':[],'score':[], 'max_depth': [],'min_samples_split': [],'min_samples_leaf': [],'random_state':[]}

    def objective(trial):
        ## HYPER PARAMETER TUNING
        max_depth = trial.suggest_int("max_depth", 2, 32)
        min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
        min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
        random_state = trial.suggest_int("random_state", 0, 50)

        # Create and fit random forest model
        model = DecisionTreeRegressor(
                max_depth=max_depth,
                min_samples_leaf=min_samples_leaf, 
                min_samples_split=min_samples_split,
                random_state=random_state
                )
        model.fit(x123_train, y123_train)
        
        # Function parameters for cross_val_score; 
        #   - estimator - The model object to use to fit the data
        #   - X - The data to fit the model on
        #   - y - The target of the model
        #   - scoring - The error metric to use
        #   - cv - The number of splits to use
        
        # error score to use; 
        #   - accuracy
        #   - balanced_accuracy
        #   - roc_auc
        #   - f1
        #   - neg_mean_absolute_error
        #   - neg_root_mean_squared_error
        #   - r2
        
        ## EVALUATION METRICS
        # you should really use cross_val_score rather than testing against the test set
        score = cross_validate(model, x123_train, y123_train, scoring=('neg_mean_absolute_percentage_error', 'r2', 'neg_root_mean_squared_error'), cv=3)      
        mean_MAPE = -1*np.mean(score['test_neg_mean_absolute_percentage_error'])
        mean_r2 = np.mean(score['test_r2'])
        mean_MSE = -1*np.mean(score['test_neg_root_mean_squared_error'])
        std_MAPE = np.std(score['test_neg_mean_absolute_percentage_error'])
        std_r2 = np.std(score['test_r2'])
        std_MSE = np.std(score['test_neg_root_mean_squared_error'])
        
        """
        y_pred_train= ridge_regressor.predict(X_test)
            
        Evaluation metrics;
        - R squared
        - MSE
        - RMSE
        - MAPE
        
        infun_MSE = mean_squared_error(y_test,y_pred_train)
        infun_RMSE = math.sqrt(infun_MSE)
        infun_r2 = r2_score(y_test, y_pred_train) 
        infun_MAPE = mean_absolute_percentage_error(y_test, y_pred_train) 
        max_coeff = ridge_regressor.coef_.round(3)
        eval_metrics_dictionary['r2'].append(infun_r2)
        eval_metrics_dictionary['MSE'].append(infun_MSE)
        eval_metrics_dictionary['RMSE'].append(infun_RMSE)
        eval_metrics_dictionary['MAPE'].append(infun_MAPE)
        """
        #coeff_used = np.sum(model.coef_!=0)

        eval_metrics_dictionary['mean_MAPE'].append(mean_MAPE)
        eval_metrics_dictionary['mean_r2'].append(mean_r2)
        eval_metrics_dictionary['mean_MSE'].append(mean_MSE)
        eval_metrics_dictionary['std_MAPE'].append(std_MAPE)
        eval_metrics_dictionary['std_r2'].append(std_r2)
        eval_metrics_dictionary['std_MSE'].append(std_MSE)
        eval_metrics_dictionary['score'].append(len(score))
        
        eval_metrics_dictionary['max_depth'].append(max_depth)
        eval_metrics_dictionary['min_samples_split'].append(min_samples_split)
        eval_metrics_dictionary['min_samples_leaf'].append(min_samples_leaf)
        eval_metrics_dictionary['random_state'].append(random_state)
        #randoforreg_dictionary['no_of_coefficients'].append(coeff_used)

        return mean_MSE*mean_MAPE*(1-mean_r2)*(1-mean_r2)
        #return mean_MSE

    study2 = optuna.create_study(study_name="DecisionTreeRegressor Optimization")
    study2.optimize(objective, n_trials=number_of_trials)
    df_framework = pd.DataFrame(eval_metrics_dictionary)
    # * by the r2_score that is closest to 1 --- you want to minimise the values of the rest, dont need to include RMSE, you overweight that value
    df_framework['combined_score'] = abs(1 - df_framework['mean_r2'])*df_framework['mean_MSE']
    return df_framework

In [209]:
def catboosterregressor_optimization(x123_train, y123_train, number_of_trials, iter_numnber):
    
    
    # input to study should be dataset and number of iterations
    eval_metrics_dictionary = {'mean_r2':[],'mean_MAPE':[],'mean_MSE':[],'std_r2':[],'std_MAPE':[],'std_MSE':[],'score':[],
                               'learning_rate' : [], 
                               'depth' : [], 
                               'l2_leaf_reg':[]
                               #'l2_leaf_reg': [], 
                               #'min_child_samples':[]
                               }

    def objective(trial):
        param = {
            'learning_rate': trial.suggest_float('learning_rate', 0.03, 0.1),
            'depth': trial.suggest_int("depth", 2, 10),
            'l2_leaf_reg': trial.suggest_float("l2_leaf_reg", 0.2, 3)
        }
        model = CatBoostRegressor(**param, iterations=iter_numnber)        
        ## EVALUATION METRICS
        # you should really use cross_val_score rather than testing against the test set        
        ## problem arises here because cross validate will fit the model each time.....
        ## MUST BUILD YOUR OWN CROSS VALIDATE SCORE, SO THAT THE MODEL IS 
        score = eric_cross_validate(model, X_train_scaled, y_train, 5)
        
        mean_MSE = np.mean(score['RMSE'])
        mean_r2 = np.mean(score['r2_score'])
        mean_MAPE = np.mean(score['MAPE'])
        std_MSE = np.std(score['RMSE'])
        std_r2 = np.std(score['r2_score'])
        std_MAPE = np.std(score['MAPE'])
                
        eval_metrics_dictionary['mean_MAPE'].append(mean_MAPE)
        eval_metrics_dictionary['mean_r2'].append(mean_r2)
        eval_metrics_dictionary['mean_MSE'].append(mean_MSE)
        eval_metrics_dictionary['std_MAPE'].append(std_MAPE)
        eval_metrics_dictionary['std_r2'].append(std_r2)
        eval_metrics_dictionary['std_MSE'].append(std_MSE)
        eval_metrics_dictionary['score'].append(len(score))
        
        eval_metrics_dictionary['learning_rate'].append(param['learning_rate'])
        eval_metrics_dictionary['depth'].append(param['depth'])
        eval_metrics_dictionary['l2_leaf_reg'].append(param['l2_leaf_reg'])

        return mean_MSE*(1-mean_r2)
        #return mean_MSE

    study2 = optuna.create_study(study_name="catboosterregressor Optimization")
    study2.optimize(objective, n_trials=number_of_trials,n_jobs=-1, timeout=24000)
    df_framework = pd.DataFrame(eval_metrics_dictionary)
    df_framework['combined_score'] = abs(1 - df_framework['mean_r2'])*df_framework['mean_MSE']
    # * by the r2_score that is closest to 1 --- you want to minimise the values of the rest, dont need to include RMSE, you overweight that value
    return df_framework

In [210]:
catboosterregressor_optimization(X_train_scaled, y_train, number_of_trials=50, iter_numnber=150)

[I 2023-09-06 09:43:27,104] A new study created in memory with name: catboosterregressor Optimization
[I 2023-09-06 09:43:29,749] Trial 0 finished with value: 57.24446826091294 and parameters: {'learning_rate': 0.09692299844184964, 'depth': 2, 'l2_leaf_reg': 0.7270451357494725}. Best is trial 0 with value: 57.24446826091294.
[I 2023-09-06 09:43:30,240] Trial 1 finished with value: 59.942316950965 and parameters: {'learning_rate': 0.059224451827714235, 'depth': 2, 'l2_leaf_reg': 2.087567433469834}. Best is trial 0 with value: 57.24446826091294.
[I 2023-09-06 09:43:30,853] Trial 8 finished with value: 51.14546747498708 and parameters: {'learning_rate': 0.07273527030190871, 'depth': 4, 'l2_leaf_reg': 1.5250930291089282}. Best is trial 8 with value: 51.14546747498708.
[I 2023-09-06 09:43:30,869] Trial 3 finished with value: 55.471688427230816 and parameters: {'learning_rate': 0.06592291440434778, 'depth': 3, 'l2_leaf_reg': 2.2235867397528457}. Best is trial 8 with value: 51.14546747498708.

,mean_r2,mean_MAPE,mean_MSE,std_r2,std_MAPE,std_MSE,score,learning_rate,depth,l2_leaf_reg,combined_score
0,0.140101,3.758278e+16,66.571130,0.045534,6.449529e+14,15.705023,3,0.096923,2,0.727045,57.244468
1,0.114560,4.002467e+16,67.697770,0.029204,6.408711e+14,16.639561,3,0.059224,2,2.087567,59.942317
2,0.200329,3.628276e+16,63.958142,0.076311,4.762573e+14,14.212279,3,0.072735,4,1.525093,51.145467
3,0.157726,3.796200e+16,65.859412,0.058272,4.956689e+14,15.538871,3,0.065923,3,2.223587,55.471688
4,0.233578,3.538809e+16,62.602329,0.094586,4.813997e+14,14.255935,3,0.098820,4,1.023096,47.979808
...,...,...,...,...,...,...,...,...,...,...,...
95,0.465894,2.866452e+16,50.541579,0.152689,5.727051e+14,6.352504,3,0.079374,10,0.299316,26.994537
96,0.476081,2.804129e+16,49.681304,0.165308,3.833481e+14,5.672475,3,0.082473,10,0.297233,26.028964
97,0.484806,2.778551e+16,49.315290,0.156572,5.947755e+14,5.362632,3,0.082472,10,0.290908,25.406956
98,0.473961,2.816469e+16,49.809103,0.161529,5.635115e+14,5.448533,3,0.083183,10,0.333795,26.201516


In [202]:
## Eric Cross Validate
def eric_cross_validate(infunc_model, x345_train, y345_train, cv):
## we want to partition the data....
    x_split = np.array_split(x345_train, cv)
    y_split = np.array_split(y345_train, cv)
    
    cross_val_dict = {'RMSE': [], 'r2_score': [], 'MAPE':[]}
    for i in list(range(len(list(x_split)))): 
        
        ## create our CV splits
        infuncXList = list(x_split)
        infuncYList = list(y_split)
        temp_X_val_split = infuncXList.pop(i)
        temp_X_train_split = np.concatenate(x_split)  
        temp_Y_val_split = infuncYList.pop(i)
        temp_Y_train_split = np.concatenate(y_split) 
        
        ## fit the model 
        inmodel = infunc_model
        inmodel.fit(temp_X_train_split, temp_Y_train_split, verbose = 0, early_stopping_rounds = 200) 
        in_ypred = inmodel.predict(temp_X_val_split)
        rmse = mean_squared_error(temp_Y_val_split, in_ypred,squared=False)
        r2 = r2_score(temp_Y_val_split, in_ypred)
        MAPE = mean_absolute_percentage_error(temp_Y_val_split, in_ypred)
        
        ## add values to the dictionary
        cross_val_dict['RMSE'].append(rmse)
        cross_val_dict['r2_score'].append(r2)
        cross_val_dict['MAPE'].append(MAPE)

    return cross_val_dict

## create a model...off the train data
## test it against the val data
eric_cross_validate(model, X_train_scaled, y_train, 5)

{'RMSE': [55.29032948644251,
  61.2763763924349,
  60.0469224945274,
  58.834256818876746,
  97.60772379226532],
 'r2_score': [0.21483019445019935,
  0.09507007871966011,
  0.09582083401021035,
  0.11442483363148614,
  0.17256356764105907],
 'MAPE': [4.110539983300202e+16,
  4.011723769756319e+16,
  4.063052359449808e+16,
  4.117907682122365e+16,
  4.12667905942553e+16]}